In [30]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import keras
from keras import layers
from keras.layers import Dense, Conv2D, MaxPooling2D, Reshape, Conv2DTranspose, UpSampling2D, Lambda
%matplotlib inline

In [40]:
# Get in dataset
data_features_test = pd.read_csv("datasets/energy-informatics-2020/csvDataFeaturesTest.csv", sep=";")

# Take out only data corresponing to attacks and remove the column labelling it as an attack
data_features_attacks = data_features_test.to_numpy()[238279:, :21]
data_features_attacks.shape
data_features_attacks = data_features_attacks.reshape(9999,21,1)

In [48]:
# Reparameterisation

def sampling(args):
    z_mean, z_log_vars = args
    batch = np.shape(z_mean)[0]
    dim = keras.backend.int_shape(z_mean)[1]
    epsilon = keras.backend.random_normal(shape=keras.backend.shape(z_mean), mean=0., stddev=1.)
    return z_mean + keras.backend.exp(0.5 * z_log_var) * epsilon


In [49]:
# Encoder

def build_encoder(input_layer, latent_dimensions, kernel_size):
    # combined convolution and maxpooling 3 times
    encoder = Conv2D(64, kernel_size, activation='relu', padding='same')(input_layer)
    encoder = MaxPooling2D((2,2))(encoder)
    encoder = Conv2D(32, kernel_size, activation='relu', padding='same')(encoder)
    encoder = MaxPooling2D((2,2))(encoder)
    encoder = Conv2D(16, kernel_size, activation='relu', padding='same')(encoder)
    encoder = MaxPooling2D((2,2))(encoder)
    encoder = Conv2D(8, kernel_size, activation='relu', padding='same')(encoder)

    # flatten data
    encoder_shape = keras.backend.int_shape(encoder)[1:]
    encoder = Reshape((np.prod(encoder_shape),))(encoder)

    # feed mean and log variance into reparameterisation function for a sampled output
    z_mean = layers.Dense(latent_dimensions, activation='linear')(encoder)
    z_log_var = layers.Dense(latent_dimensions, activation='linear')(encoder)
    # reparametrisation
    z = Lambda(sampling, output_shape=(latent_dimensions,))([z_mean, z_log_var])

    model = keras.Model(input_layer, [z_mean, z_log_var, z], name='encoder')
    return model, z_mean, z_log_var, encoder_shape


In [50]:
# Decoder

def build_decoder(latent_dimensions, kernel_size, encoder_shape, input_shape):
    lats = keras.Input(shape=(latent_dimensions,))
    #reshape latent space back into shape of processed data and flatten
    decoder = Dense(np.prod(encoder_shape), activation='relu')(lats)
    decoder = Reshape(encoder_shape)(decoder)

    #layer then convolved and upsampled in reverse order of encoder to product data the same shape as the original input
    decoder = Conv2DTranspose(16, kernel_size, activation='relu', padding='same')(decoder)
    decoder = UpSampling2D((2,2))(decoder)
    decoder = Conv2DTranspose(32, kernel_size, activation='relu', padding='same')(decoder)
    decoder = UpSampling2D((2,2))(decoder)
    decoder = Conv2DTranspose(64, kernel_size, activation='relu', padding='same')(decoder)
    decoder = UpSampling2D((2,2))(decoder)

    outputs = Conv2DTranspose(3, kernel_size, activation='relu', padding='same')(decoder)

    model = keras.Model(lats, outputs, name='decoder')
    return model

In [51]:
# Autoencoder

def build_autoencoder(encoder, decoder, input_layer, z_mean, z_log_var):
    output_layer = decoder(encoder(input_layer)[2])
    vae = keras.Model(input_layer, output_layer, name='autoencoder')

    reconstruction_loss = tf.reduce_mean(1000 * tf.square(input_layer-output_layer))

    kl_loss = -0.5 * keras.backend.sum(1 + z_log_var - keras.backend.square(z_mean) - keras.backend.exp(z_log_var), axis=1)
    vae_loss = tf.reduce_mean(reconstruction_loss + kl_loss)

    vae.add_loss(vae_loss)
    vae.add_metric(tf.reduce_sum(1000*tf.square(input_layer - output_layer)), aggregation='mean')
    vae.add_metric(kl_loss, aggregation='mean')
    vae.compile(optimizer='adam')

    return vae, reconstruction_loss 

In [52]:
model_name = "test"
input_shape = data_features_attacks.shape
latent_dimensions = 200
batch_size = 128
epochs = 1
kernel_size = 3
input_layer = keras.Input(shape=input_shape)

encoder, z_mean, z_log_var, encoder_shape = build_encoder(input_layer, latent_dimensions, kernel_size)
decoder = build_decoder(latent_dimensions, kernel_size, encoder_shape, input_shape)
autoencoder, reconstruction_loss = build_autoencoder(encoder, decoder, input_layer, z_mean, z_log_var)

autoencoder.fit(data_features_attacks, epochs=epochs, batch_size=batch_size, shuffle=True)
autoencoder.save_weights(model_name+'.h5')

NameError: Exception encountered when calling layer "lambda" (type Lambda).

name 'z_log_var' is not defined

Call arguments received by layer "lambda" (type Lambda):
  • inputs=['tf.Tensor(shape=(None, 200), dtype=float32)', 'tf.Tensor(shape=(None, 200), dtype=float32)']
  • mask=None
  • training=None